In [1]:
pip install shared_utils

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import shared_utils
import pandas as pd
import geopandas as gpd
import re
import google.auth
import os
import gcsfs

from calitp_data_analysis import get_fs
from calitp_data_analysis import geography_utils, utils
from calitp_data_analysis.sql import get_engine

fs = get_fs()
db_engine = get_engine()

In [ ]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses'

In [ ]:
# Querying dim organizations dataset to get ridership data 
with db_engine.connect() as connection:
    query = """
        SELECT
            key, ntd_id, upt, agency, period_year
        FROM 
            cal-itp-data-infra.mart_ntd.dim_monthly_ridership_with_adjustments
    """
    dim_ridership= pd.read_sql(query, connection)

In [ ]:
dim_ridership.head(5)

In [ ]:
# Load the stored organization dataset from the specified GCS file path.
with fs.open(f"{GCS_FILE_PATH}/transit_provider_dashboard/agency_level_census_data.parquet.parquet", "rb") as f:
    agency_acs = pd.read_parquet(f)